## TD Learning

Implement Sarsa(Lamda) in 21s. Initialise the value function to zero. Use the same step-szie and exploration schedules as in the previous section. Run the algorithm with parameter values Lamda = {0, 0.1, 0.2, ..., 1}. Stop each run after 1000 episodes and report the mean-squared error Sigma_s,a(Q(s, a) - Q*(s, a))^square over all states s and actions a, comparing the true values Q*(s, a) computed in the previous section with the estimated values Q(s, a) computed by Sarsa. Plot the mean-squared error against Lamda. For Lamda = 0 and Lamda = 1 only, plot the learning curve of mean-squared error against episode number.

__Value Function__
- Initialise : zero

__step-size__
- alpha_t = 1/N(s_t, a_t)

__epsilon-greedy exploration__
- epsilon_t = N_0/(N_0 + N(s_t)), N_0 = 100 is a constant

N(s): number of times that state __s__ has been visited.  
N(s, a): number of times that action __a__ has been selected from state s.

__Lamda__
- {0, 0.1, 0.2, ..., 1}

__Rule__
- Stop each run after 1000 episodes and report the mean-squared error

__MSE__
- Sigma_s,a(Q(s,a)-Q*(s,a))^square over all states s and actions a
- comparing the true values Q*(s,a) computed in the previous section with estimated values Q(s,a) computed by Sarsa.

__Optimal value function__  
 - V\*(s) = max_aQ\*(s, a) 
 - computed bt 
  
Plot the mean-squared error against Lamda.  
For Lamda = 0 and Lamda = 1 only, plot the learning curve of mean-squared error against episode number.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [11]:

class EasyEnv(object):
    def __init__(self):
        self.lowerbound = 1
        self.upperbound = 21
        # 1 is hit and 0 is stick
        self.actions = [0, 1]
    
    def initGame(self):
        self.playerValue = np.random.randint(1, 11)
        self.dealerValue = np.random.randint(1, 11)
        
    def draw(self):
        card_value = np.random.randint(1, 11)
        
        if round(np.random.rand(), 2) <= 0.3:
            return -card_value
        else:
            return card_value
        
    def get_state(self):
        return self.playerValue, self.dealerValue
    
    def step(self, playerValue, dealerValue, action):
        
        # Hit
        if action == 1:
            playerValue += self.draw()
            
            if playerValue > self.upperbound or playerValue < self.lowerbound:
                reward = -1
                terminated = True
            else:
                reward = 0
                terminated = False
        else:
            # Player Action is Stick. Dealer`s turn.
            while dealerValue < 17:
                dealerValue += self.draw()
                
            if dealerValue > self.upperbound or dealerValue < self.lowerbound or playerValue > dealerValue:
                reward = 1
            elif playerValue == dealerValue:
                reward = 0
            else:
                reward = -1
                
            terminated = True
        
        return playerValue, dealerValue, reward, terminated
    

In [12]:

N_0 = 100
actions = [0, 1] # 0 is Stick, 1 is Hit.
N_s_dict = {} # Number of times that state s has been visited.
N_sa_dict = {} # Number of times that action a has been selected from state s.
Q_sa_dict = {} # Action-State Value
V_s_dict = {} # State Value

   
def calc_epsilon(N_s):
    if N_s not in N_s_dict.keys():
        N_s_dict[N_s] = 0
    epsilon_t = N_0/(N_0 + N_s_dict[N_s])
    return epsilon_t

def calc_alpha(N_sa):
    alpha = 1 / N_sa
    return alpha

def epsilonGreedy(pValue, dValue):
    epsilon = calc_epsilon((pValue, dValue))
    if (pValue, dValue, 0) not in Q_sa_dict.keys():
        Q_sa_dict[(pValue, dValue, 0)] = 0
    if (pValue, dValue, 1) not in Q_sa_dict.keys():
        Q_sa_dict[(pValue, dValue, 1)] = 0
  
    max_action = np.argmax([Q_sa_dict[pValue, dValue, act] for act in actions])
    
    #RandomPolicy
    if epsilon is 1:
        return np.random.choice(actions)
    
    # Exploitation
    if round(np.random.rand(), 2) > epsilon:
        return max_action
    # Explore
    else:
        if max_action:
            return 0
        else:
            return 1

env = EasyEnv()
episodes = 1000000

for episode in range(episodes):
    terminated = False
    H = []

    env.initGame()
    pValue, dValue = env.get_state()
    reward = 0

    while not terminated:

        if (pValue, dValue) not in N_s_dict.keys():
            N_s_dict[(pValue, dValue)] = 0
        N_s_dict[(pValue, dValue)] += 1

        action = epsilonGreedy(pValue, dValue)
        if (pValue, dValue, action) not in N_sa_dict.keys():
            N_sa_dict[(pValue, dValue, action)] = 1
        else:
            N_sa_dict[(pValue, dValue, action)] += 1

        pPrime, dPrime, reward, terminated = env.step(pValue, dValue, action)

        H.append([pValue, dValue, action, reward])

        pValue, dValue = pPrime, dPrime

    G = reward

    for (pValue, dValue, action, _) in H:
        alpha = calc_alpha(N_sa_dict[(pValue, dValue, action)])
        if (pValue, dValue, action) not in Q_sa_dict.keys():
            Q_sa_dict[(pValue, dValue, action)] = 0
        Q_sa_dict[(pValue, dValue, action)] += alpha*(G - Q_sa_dict[(pValue, dValue, action)])

In [13]:
Q_sa_dict

{(1, 1, 0): -0.5370585952658135,
 (1, 1, 1): -0.7182164475959877,
 (1, 2, 0): -0.5277199621570463,
 (1, 2, 1): -0.5992063492063493,
 (1, 3, 0): -0.5308739396330652,
 (1, 3, 1): -0.701487925165127,
 (1, 4, 0): -0.5121027990835747,
 (1, 4, 1): -0.6622615252972074,
 (1, 5, 0): -0.48437349745167935,
 (1, 5, 1): -0.6612026820073829,
 (1, 6, 0): -0.4713856980233885,
 (1, 6, 1): -0.6851404617568617,
 (1, 7, 0): -0.4980824073163537,
 (1, 7, 1): -0.6420824295010847,
 (1, 8, 0): -0.525563983843956,
 (1, 8, 1): -0.6726396882167477,
 (1, 9, 0): -0.5679651885619563,
 (1, 9, 1): -0.6729943064182191,
 (1, 10, 0): -0.5769010863350469,
 (1, 10, 1): -0.6926315789473682,
 (2, 1, 0): -0.5302352712424658,
 (2, 1, 1): -0.6717770096424855,
 (2, 2, 0): -0.5153881915388235,
 (2, 2, 1): -0.6311458793059078,
 (2, 3, 0): -0.517149135377855,
 (2, 3, 1): -0.644171779141104,
 (2, 4, 0): -0.5106985832926263,
 (2, 4, 1): -0.6429922531174551,
 (2, 5, 0): -0.5022933541524346,
 (2, 5, 1): -0.6312670765027321,
 (2, 6, 0):

In [40]:
   
def calc_epsilon(N_s):
    if N_s not in N_s_dict.keys():
        N_s_dict[N_s] = 0
    epsilon_t = N_0/(N_0 + N_s_dict[N_s])
    return epsilon_t

def calc_alpha(N_sa):
    alpha = 1 / N_sa
    return alpha

def epsilonGreedy(pValue, dValue):
    epsilon = calc_epsilon((pValue, dValue))
    if (pValue, dValue, 0) not in Q_sa_dict.keys():
        Q_sa_dict[(pValue, dValue, 0)] = 0
    if (pValue, dValue, 1) not in Q_sa_dict.keys():
        Q_sa_dict[(pValue, dValue, 1)] = 0
  
    max_action = np.argmax([Q_sa_dict[pValue, dValue, act] for act in actions])
    
    #RandomPolicy
    if epsilon is 1:
        return np.random.choice(actions)
    
    # Exploitation
    if round(np.random.rand(), 2) > epsilon:
        return max_action
    # Explore
    else:
        if max_action:
            return 0
        else:
            return 1

env = EasyEnv()
episodes = 1000
lmds = np.arange(0,11)/10
mselamdas = np.zeros((len(lmds), episodes))
finalMSE = np.zeros(len(lmds))
T_Q = Q_sa_dict
N_0 = 100
actions = [0, 1] # 0 is Stick, 1 is Hit.


for lamC, lmd in enumerate(lmds):

    N_s_dict = {} # Number of times that state s has been visited.
    N_sa_dict = {} # Number of times that action a has been selected from state s.
    Q_sa_dict = {} # Action-State Value
    V_s_dict = {} # State Value
    
    for i in range(1, 22):
        for j in range(1, 11):
            Q_sa_dict[(i, j, 0)] = 0
            Q_sa_dict[(i, j, 1)] = 0
    
    wins = 0

    for episode in range(episodes):

        terminated = False
        E = {}

        env.initGame()
        pValue, dValue = env.get_state()
        action = epsilonGreedy(pValue, dValue)
        SA = []
        reward = 0

        while not terminated:

            pPrime, dPrime, reward, terminated = env.step(pValue, dValue, action)
            
            if not terminated:
                aPrime = epsilonGreedy(pPrime, dPrime)
                tdError = reward + Q_sa_dict[(pPrime, dPrime, aPrime)] - Q_sa_dict[pValue, dValue, action]
            else:
                tdError = reward - Q_sa_dict[(pValue, dValue, action)]
            
            if (pValue, dValue, action) not in E.keys():
                E[(pValue, dValue, action)] = 0
            E[(pValue, dValue, action)] += 1
            
            if (pValue, dValue, action) not in N_sa_dict.keys():
                N_sa_dict[(pValue, dValue, action)] = 1
            else:
                N_sa_dict[(pValue, dValue, action)] += 1
            
            if (pValue, dValue) not in N_s_dict.keys():
                N_s_dict[(pValue, dValue)] = 0
            N_s_dict[(pValue, dValue)] += 1
            
            SA.append([pValue, dValue, action])
            
            alpha = calc_alpha(N_sa_dict[(pValue, dValue, action)])
            for (_p, _d, _a) in SA:
                Q_sa_dict[(_p, _d, _a)] += alpha*tdError*E[_p, _d, _a]
                E[_p, _d, _a] *= lmd
                
            if not terminated:
                pValue, dValue, action = pPrime, dPrime, aPrime

        if reward == 1:
            wins += 1
        
        mse = np.sum(np.square(np.array(list(Q_sa_dict.values())) - np.array(list(T_Q.values())))) / (21*10*2)
        
        mselamdas[lamC, episode] = mse
        
        if (episode + 1) % 1000 == 0:
            print("Lamda=%.1f Episode %06d, MSE %5.3f, Wins %.3f"%(lmd, episode+1, mse, wins/(episode+1)))

    finalMSE[lamC] = mse
    


Lamda=0.0 Episode 001000, MSE 1.420, Wins 0.099
Lamda=0.1 Episode 001000, MSE 1.367, Wins 0.108
Lamda=0.2 Episode 001000, MSE 1.336, Wins 0.110
Lamda=0.3 Episode 001000, MSE 1.355, Wins 0.108
Lamda=0.4 Episode 001000, MSE 1.353, Wins 0.122
Lamda=0.5 Episode 001000, MSE 1.322, Wins 0.103
Lamda=0.6 Episode 001000, MSE 1.239, Wins 0.121
Lamda=0.7 Episode 001000, MSE 1.231, Wins 0.103
Lamda=0.8 Episode 001000, MSE 1.182, Wins 0.078
Lamda=0.9 Episode 001000, MSE 1.465, Wins 0.079
Lamda=1.0 Episode 001000, MSE 1.674, Wins 0.099


In [42]:
mselamdas

array([[1.84699349, 1.83936787, 1.83285338, ..., 1.42015601, 1.42004299,
        1.41993344],
       [1.83676824, 1.83427444, 1.83066262, ..., 1.36708175, 1.3668824 ,
        1.36672454],
       [1.84395148, 1.85114392, 1.84043411, ..., 1.33582403, 1.33556361,
        1.33566465],
       ...,
       [1.82638261, 1.81633866, 1.81318692, ..., 1.18256302, 1.18251181,
        1.18237809],
       [1.82261849, 1.77290277, 1.72998906, ..., 1.46608368, 1.46529003,
        1.46514453],
       [1.83466331, 1.81597745, 1.80828299, ..., 1.67464582, 1.67454753,
        1.67400519]])